In [1]:
import pygame
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import random

from keras.layers import Dense,Input,Flatten
from keras.models import Model
from keras.utils import to_categorical

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


In [2]:
def modelRoot(inputX,classes):
    X=Flatten()(inputX)
    X=Dense(8,activation='relu')(X)
    X=Dense(classes,activation=None)(X)
    return X

In [3]:
inputShape= Input((3,1))
mRoot=modelRoot(inputShape,2)
model=Model(inputs=inputShape,outputs=mRoot)
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 1)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 50
Trainable params: 50
Non-trainable params: 0
_________________________________________________________________


In [4]:
from keras.models import model_from_json

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("modelWeights.h5")

In [5]:
def nextAction(dataQ,explorationRate,willLearn):
    alives=dataQ.shape[0]
    ans=np.zeros(alives)
    for bb in range(alives):
        if random.random() > explorationRate or not willLearn:
            ans[bb]=np.argmax(dataQ[bb])
        else:
            rn=random.random()
            if rn>0.5:
                ans[bb]=0
            else :
                ans[bb]=1
    return ans

In [6]:
iterations=100
explorationRate=1.0
explorationDelta=1.0/iterations
lamda=0.95
showGame=True
willLearn=False
out=False
for gen in range(iterations):
    winHeight=720
    winWidth=350
    totalBalls=100
    posX=np.zeros(totalBalls)
    posY=np.zeros(totalBalls)
    reward=np.zeros(totalBalls)
    justNow=np.zeros(totalBalls)
    ballAlive=np.ones(totalBalls)
    score=np.zeros(totalBalls)
    unit=3
    isJump=0
    direction=1
    jumpDuration=12
    jumpCount=-jumpDuration
    if showGame:
        CC=pygame.time.Clock()

    for it in range(100):
        rn=random.random()
        if rn>0.5:
            posX[it]=140
        else :
            posX[it]=210
        posY[it]=650

    totalBoxes=8
    boxX=np.zeros(totalBoxes)
    boxY=np.zeros(totalBoxes)
    temp=100
    passed=np.zeros((totalBalls,totalBoxes))
    for it in range(totalBoxes):
        rn=random.random()
        if rn>0.5:
            boxX[it]=140
        else :
            boxX[it]=210

        boxY[it]=temp
        temp-=200

    if showGame:
        backImage=pygame.image.load('images/back.jpg')
        pygame.init()
        pygame.font.init() 
        myfont = pygame.font.SysFont('Comic Sans MS', 12)
        win=pygame.display.set_mode((winWidth,winHeight))
        pygame.display.set_caption("Jump the wall")

    done=False
    while not done:

        if showGame:
            CC.tick(60)
            win.fill((255,255,255))
            for event in pygame.event.get():
                if event.type==pygame.QUIT:
                    done=True
                    out=True
            keys=pygame.key.get_pressed()
            for it in range(totalBalls):
                if keys[pygame.K_LEFT] and posX[it]>150:
                    posX[it]=140
                if keys[pygame.K_RIGHT] and posX[it]<150:
                    posX[it]=210
            pygame.draw.rect(win,(0,0,0),(90,0,10,720))
            pygame.draw.rect(win,(0,0,0),(250,0,10,720))

        ########################## reinforcement learning old state info
        alives=0
        oldStates=[]
        for it in range(totalBalls):
            if ballAlive[it]==0:
                continue
            alives+=1
            reward[it]=0
            if posX[it]<150:
                tX=0
            else :
                tX=1
            #############
           
            mn=10000
            for it2 in range(totalBoxes):
                temp1=(posY[it]+30)-(boxY[it2]-30)
                if temp1>=0 and temp1<mn:
                    mn=temp1
                    dis=boxY[it2]-posY[it]
                    if boxX[it2]<150:
                        bX=0
                    else:
                        bX=1

          
            #############
            oldState=[]
            oldState.append(dis/500.0)
            oldState.append(bX)
            oldState.append(tX)
            oldState=np.asarray(oldState)
            oldStates.append(oldState)
        oldStates=np.asarray(oldStates)
        oldStates=oldStates.reshape(oldStates.shape[0],oldStates.shape[1],1)
        oldQs=model.predict(oldStates)
        ############  next Action
        action=nextAction(oldQs,explorationRate,willLearn)
        ########################## 

        ##applying actions
        cur=0
        for it in range(totalBalls):
            if ballAlive[it]==1:
                if action[cur]==0 and posX[it]>150:
                    posX[it]=140
                if action[cur]==1 and posX[it]<150:
                    posX[it]=210
                cur+=1

        #updating positions
        mx=100
        for it in range(totalBoxes):
            boxY[it]+=unit
            mx=min(mx,boxY[it])

        for it in range(totalBoxes):
            if boxY[it] >= 800:
                for it2 in range(totalBalls):
                    passed[it2,it]=0
                boxY[it] = mx-200
                rn=random.random()
                if rn>0.5:
                    boxX[it]=140
                else :
                    boxX[it]=210
                break

        ##need to update reward,justnow,points
        #intersections
        for it in range(totalBalls):
            if ballAlive[it]==0:
                continue
            for it2 in range(totalBoxes):
                dX=posX[it]-boxX[it2]
                dY=posY[it]-boxY[it2]
                dd=dX*dX+dY*dY
                dd=np.sqrt(dd)
                if dd<60:
                    ballAlive[it]=0
                    justNow[it]=1
                    reward[it]=-1
                    break
                if dY<-60 and passed[it,it2]==0:
                    passed[it,it2]=1
                    score[it]+=1
                    reward[it]=1

        ########################## reinforcement learning new state info
        newStates=[]
        for it in range(totalBalls):
            if ballAlive[it]==0 and justNow[it]==0:
                continue
            if posX[it]<150:
                tX=0
            else :
                tX=1
            #############
            mn=10000
            for it2 in range(totalBoxes):
                temp1=(posY[it]+30)-(boxY[it2]-30)
                if temp1>=0 and temp1<mn:
                    mn=temp1
                    dis=boxY[it2]-posY[it]
                    if boxX[it2]<150:
                        bX=0
                    else:
                        bX=1
            #############
            newState=[]
            newState.append(dis/500.0)
            newState.append(bX)
            newState.append(tX)
            newState=np.asarray(newState)
            newStates.append(newState)
        newStates=np.asarray(newStates)
        newStates=newStates.reshape(newStates.shape[0],newStates.shape[1],1)
        newQs=model.predict(newStates)
        ##########################

        ########################## reinforcement learning train
        cur=0

        for it in range(totalBalls):
            if ballAlive[it]==0 and justNow[it]==0:
                continue
            okk=reward[it] + lamda * np.amax(newQs[cur])
            oldQs[cur,int(action[cur])] = okk
            cur+=1
        if willLearn:
            model.fit(oldStates,oldQs,verbose=0)

        ##########################


        #drawing balls and boxes
        if showGame:
            for it in range(totalBalls):
                if ballAlive[it]==1 or justNow[it]==1:
                    pygame.draw.circle(win,(255,0,0),(int(posX[it]),int(posY[it])),30)

            for it in range(totalBoxes):
                pygame.draw.circle(win,(0,0,255),(int(boxX[it]),int(boxY[it])),30)

        highest=0
        for it in range(totalBalls):
            highest=max(highest,score[it])

        #updating justNow
        for it in range(totalBalls):
            justNow[it]=0

        #updating score
        alives=totalBalls
        for it in range(totalBalls):
            if ballAlive[it]==0:
                alives-=1

        left=0
        right=0
        for it in range(totalBalls):
            if ballAlive[it]==1:
                if posX[it]<150:
                    left+=1
                else :
                    right+=1

        if showGame:
            text=myfont.render('Score: '+str(highest),1,(0,0,0))
            text2=myfont.render('Alives: '+str(alives),1,(0,0,0))
            text3=myfont.render('Left: '+str(left),1,(0,0,0))
            text4=myfont.render('Right: '+str(right),1,(0,0,0))
            text5=myfont.render('Gen: '+str(gen+1),1,(0,0,0))

            win.blit(text5,(10,10))
            win.blit(text,(270,10))
            win.blit(text2,(270,50))
            win.blit(text3,(270,90))
            win.blit(text4,(270,120))

            pygame.display.update()



        #checking alives
        allDead=0
        for it in range(totalBalls):
            if ballAlive[it]==0:
                allDead+=1
        if allDead==totalBalls:
            done=True
    if explorationRate>0:
        explorationRate-=explorationDelta
    print(str(gen)+" --> "+str(highest))
    if out==True:
        break
        
if showGame:
    pygame.quit()

0 --> 75.0
